# Proyecto 1. Obtención y transformación de datos sobre población, vivienda, pobreza y delitos en Sonora
### *Maestría en Ciencia de Datos - Ingeniería de Características*

El objetivo del projecto es desarrollar un programa que ayude a descargar datos a partir de diferentes APIs, o usando un método para descargas masivas, para su posterior transformación y formateo.

La presente libreta toma archivos descargados con el script `obtencion_datos.py`, y transforma los datos para ser utilizados según su tipo, seleccionando las características que se consideraron útiles. En un principio se generan tres DataFrames diferentes pero se unen en uno solo que contiene toda la información.

Finalmente, se genera un diccionario de datos apartir de dicho DataFrame.

In [1]:
import pandas as pd

## Lectura de dataframes

In [2]:
# Carga archivo excel de población y vivienda en un df
ccpv_df = pd.read_excel('datasets/ccpv_INEGI/cpv_26.xlsx', sheet_name=0, header = 4)
# Carga archivo excel de delitos en un df
del_df = pd.read_excel('datasets/del_INEGI/delitos_26.xlsx', sheet_name=0, header = 0)
# Carga archivo csv de pobreza en un df
po_df = pd.read_csv('datasets/po_API/po_API.csv', index_col=0)

## Limpieza y selección de dataframes

Haciendo un análisis muy general de la cantidad de datos faltantes por año en cada dataset, se determinó utilizar los datos correspondientes al 2010 para el `ccpv_df` y `po_df`. En el caso de `del_df` no se contaban con datos de ese año, por lo que se seleccionó aquel con minoría de datos faltantes más cercano al 2010: con una diferencia de dos años, 2012.

In [13]:
del_df.isna().sum()

entidad              0
desc_entidad         0
municipio            0
desc_municipio       0
id_indicador         0
indicador            0
2011              4969
2012              1251
2013              3438
2014              3414
2015              4032
unidad_medida        0
dtype: int64

In [3]:
# Genera un dataframe de 'seguridad' para no sobreescribir el original
ccpv_ty = ccpv_df.copy(deep = True)
# Se omiten los registros  de datos generales que corresponden a todo el estado (cve_municipio = 0 para todo Sonora)
ccpv_ty = ccpv_ty[ccpv_ty['cve_municipio'] != 0]
# Se omiten los registros (indicadores) de los cuales no hay datos para el 2010
ccpv_ty = ccpv_ty[ccpv_ty['2010'].notna()]
# Selección de indicadores de interés
ccpv_ty = ccpv_ty[ccpv_ty['indicador'].isin(['Población total','Población total hombres',
       'Población total mujeres','Edad mediana','Población en hogares censales',
       'Población en hogares familiares','Población en hogares no familiares',
       'Hogares censales','Hogares con jefatura masculina','Hogares con jefatura femenina',
       'Tamaño promedio de los hogares censales','Tamaño promedio de los hogares con jefe hombre',
       'Tamaño promedio de los hogares con jefe mujer','Relación hombres-mujeres',
       'Porcentaje de personas de 15 años y más alfabetas',
       'Promedio de ocupantes en viviendas particulares habitadas',
       'Viviendas particulares habitadas que disponen de energía eléctrica',
       'Viviendas particulares habitadas que disponen de drenaje',
       'Viviendas particulares habitadas que disponen de excusado o sanitario',
       'Viviendas particulares habitadas que disponen de computadora',
       'Población derechohabiente a servicios de salud',
       'Grado promedio de escolaridad de la población de 15 y más años',
       'Población de 5 años y más hablante de lengua indígena',
       'Porcentaje de viviendas con agua entubada dentro de la vivienda'])]
# Diccionario auxiliar para la generación del diccionario de datos
dict_aux = {}
for index, row in ccpv_ty.iterrows():
    dict_aux.update({row.indicador:row.unidad_medida})
# Tranformación de indicadores (columna) a características (pivot long to wide)
ccpv_ty = ccpv_ty.pivot(index=['cve_municipio'], columns='indicador', values='2010')\
    .rename_axis(columns = None).reset_index()
ccpv_ty.head(3)

,cve_municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Hogares con jefatura femenina,Hogares con jefatura masculina,Población de 5 años y más hablante de lengua indígena,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,Porcentaje de viviendas con agua entubada dentro de la vivienda,Promedio de ocupantes en viviendas particulares habitadas,Relación hombres-mujeres,Tamaño promedio de los hogares censales,Tamaño promedio de los hogares con jefe hombre,Tamaño promedio de los hogares con jefe mujer,Viviendas particulares habitadas que disponen de computadora,Viviendas particulares habitadas que disponen de drenaje,Viviendas particulares habitadas que disponen de energía eléctrica,Viviendas particulares habitadas que disponen de excusado o sanitario
0,1,30.0,7.81,703.0,91.0,612.0,1.0,2096.0,2637.0,2556.0,...,92.603129,3.75,114.04,3.75,3.85,3.09,153.0,670.0,694.0,679.0
1,2,24.0,8.80,20185.0,5333.0,14852.0,440.0,48871.0,77516.0,75075.0,...,91.984714,3.84,102.81,3.84,3.95,3.53,6743.0,19767.0,19482.0,19868.0
2,3,28.0,6.94,6628.0,1218.0,5410.0,1264.0,21303.0,25819.0,25056.0,...,30.350136,3.90,109.28,3.90,4.03,3.30,789.0,3055.0,6034.0,5632.0


In [4]:
# Genera un dataframe de 'seguridad' para no sobreescribir el original
del_ty = del_df.copy(deep = True)
# Se omiten los registros  de datos generales que corresponden a todo el estado (municipio = 0 para todo Sonora)
del_ty = del_ty[del_ty['municipio'] != 0]
# Se omiten los registros (indicadores) de los cuales no hay datos para el 2012
del_ty = del_ty[del_ty['2012'].notna()]
# Selección de delitos registrados (no tentativa)
del_ty = del_ty[del_ty['indicador'].str.startswith('Presuntos delitos registrados')]
# Diccionario auxiliar para la generación del diccionario de datos
for index, row in del_ty.iterrows():
    dict_aux.update({row.indicador:row.unidad_medida})
# Tranformación de indicadores (columna) a características (pivot long to wide)
del_ty = del_ty.pivot(index=['municipio'], columns='indicador', values='2012')\
    .rename_axis(columns = None).reset_index()
# Cambio de nombre al id de los minicipios para posterior merge
del_ty.rename(columns={'municipio':'cve_municipio'}, inplace=True)
del_ty.head(3)

,cve_municipio,Presuntos delitos registrados como abuso de confianza en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como abuso sexual en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como allanamiento de morada en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como amenazas en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como corrupción de menores en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como daño a la propiedad en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como despojo en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como evasión de presos en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como extorsión en las intervenciones de la policía municipal (fuero común),...,Presuntos delitos registrados como robo en transporte público colectivo en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como robo en transporte público individual en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como secuestro en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como secuestro exprés en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como trata de personas en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como tráfico de menores en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como violación equiparada en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como violación simple en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados como violencia familiar en las intervenciones de la policía municipal (fuero común),Presuntos delitos registrados en las intervenciones de la policía municipal
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,2,0,16,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,25,142,934
2,3,0,2,1,1,0,24,3,0,0,...,2,0,0,0,0,0,0,1,7,585


In [5]:
# Genera un dataframe de 'seguridad' para no sobreescribir el original
po_ty = po_df.copy(deep = True)
# Se omiten los registros (indicadores) de los cuales no hay datos para el 2010
po_ty = po_ty[po_ty['Year']==2010]
# Se formatea el id de los municipios a lo últimos dos dígitos para que compatibilice con el resto de df 
po_ty['Municipality ID'] = po_ty['Municipality ID']-26000
# Cambio de nombre al id de los minicipios para posterior merge
po_ty.rename(columns={'Municipality ID':'cve_municipio'}, inplace=True)
# Omición de las columnas 'Municipality' y 'Year'
po_ty = po_ty.loc[:, ~po_ty.columns.isin(['Municipality', 'Year'])]
po_ty.head(3)

,cve_municipio,Poverty,Extreme Poverty,Moderate Poverty,Population No Vulnerable,Educational Backwardness,Deprivation Quality Housing Spaces,Deprivation Health Services,Deprivation Social Security,Deprivation Basic Services Housing,Deprivation Food Access,Population with at least 1 Social Lack,Population with at least 3 Social Lacks,Income below Welfare Line,Income below Min Welfare Line
0,1,787.0,58.0,729.0,499.0,588.0,321.0,533.0,1739.0,116.0,248.0,2018.0,354.0,908.0,288.0
3,2,29013.0,3338.0,25675.0,18619.0,11448.0,6654.0,27272.0,41409.0,4086.0,12977.0,54954.0,10974.0,35544.0,9375.0
6,3,17674.0,5569.0,12105.0,1525.0,7116.0,7161.0,3834.0,21938.0,16751.0,15151.0,25456.0,15680.0,18196.0,7058.0


## Combinación de dataframes

Como el formato de los tres dataframes ya es '*tidy*' y el *id* de sus municipio es el mismo, se utilizó un *merge* tipo *outer* para garantizar que no se pierda ningún dato. 

In [6]:
son_df = ccpv_ty.merge(del_ty, how='outer', on='cve_municipio')
son_df = son_df.merge(po_ty, how='outer', on='cve_municipio')
son_df.columns

Index(['cve_municipio', 'Edad mediana',
       'Grado promedio de escolaridad de la población de 15 y más años',
       'Hogares censales', 'Hogares con jefatura femenina',
       'Hogares con jefatura masculina',
       'Población de 5 años y más hablante de lengua indígena',
       'Población derechohabiente a servicios de salud',
       'Población en hogares censales', 'Población en hogares familiares',
       'Población en hogares no familiares', 'Población total',
       'Población total hombres', 'Población total mujeres',
       'Porcentaje de personas de 15 años y más alfabetas',
       'Porcentaje de viviendas con agua entubada dentro de la vivienda',
       'Promedio de ocupantes en viviendas particulares habitadas',
       'Relación hombres-mujeres', 'Tamaño promedio de los hogares censales',
       'Tamaño promedio de los hogares con jefe hombre',
       'Tamaño promedio de los hogares con jefe mujer',
       'Viviendas particulares habitadas que disponen de computadora',
 

Como el título de algunos de los indicadores/características/columnas es muy largo, se prefirío darles un formato común con el sufijo 'ind_' y un entero según su posición.

In [7]:
# Generación de dictionaros con los id de los municipios y su correspondiente, y de los indicadores
mun_dict = {}
ind_dict = {}
for index, row in ccpv_df.iterrows():
    mun_dict.update({row.desc_municipio:row.cve_municipio})
for col in son_df.columns:
    idx = son_df.columns.get_loc(col)
    if col != 'cve_municipio':
        ind_dict.update({'ind_'+str(idx):col})
# Cambio de nombre a todas las columnas excepto 'cve_municipio'
for idx in ind_dict:
    son_df.rename(columns={ind_dict[idx] : idx}, inplace=True)
son_df.head(3)

,cve_municipio,ind_1,ind_2,ind_3,ind_4,ind_5,ind_6,ind_7,ind_8,ind_9,...,ind_72,ind_73,ind_74,ind_75,ind_76,ind_77,ind_78,ind_79,ind_80,ind_81
0,1,30.0,7.81,703.0,91.0,612.0,1.0,2096.0,2637.0,2556.0,...,588.0,321.0,533.0,1739.0,116.0,248.0,2018.0,354.0,908.0,288.0
1,2,24.0,8.80,20185.0,5333.0,14852.0,440.0,48871.0,77516.0,75075.0,...,11448.0,6654.0,27272.0,41409.0,4086.0,12977.0,54954.0,10974.0,35544.0,9375.0
2,3,28.0,6.94,6628.0,1218.0,5410.0,1264.0,21303.0,25819.0,25056.0,...,7116.0,7161.0,3834.0,21938.0,16751.0,15151.0,25456.0,15680.0,18196.0,7058.0


## Exportar el dataframe

In [8]:
son_df.to_parquet('sonora_df.parquet', index=False, engine= 'pyarrow')

## Generación de diccionario de datos

In [9]:
# Se genera el dataframe a partir de la tabla dtypes de cada columna
dict_df = son_df.dtypes.to_frame('Dtypes').reset_index()
dict_df.rename(columns={'index':'Variable'}, inplace=True)
dict_df['Descripcion'] = ''
dict_df['Unidades'] = 'No especificado'
dict_df['Programa'] = 'Consejo Nacional de Evaluacion de la Politica de Desarrollo Social 2010 (DataMexico)'
# Se llena los valores con los datos de df previos
for index, row in dict_df.iterrows():
    if index < 25:
        row.Programa = 'Censo de Poblacion y Vivienda 2010 (INEGI)'
    elif 24 < index < 79:
        row.Programa = 'Censo Nacional de Imparticion de Justicia Estatal 2012 (INEGI)'
    if row.Variable == 'cve_municipio':
        row.Descripcion = 'ID de municipio sonorense (Revisar pestaña "Municipios")'
    for key in ind_dict:
        if row.Variable == key: 
            row.Descripcion = ind_dict[key]
    for key in dict_aux:
        if row.Descripcion == key:
            row.Unidades = dict_aux[key]
# ExcelWriter permite escribir diferntes sheets en un solo xlsx
dicc_datos = pd.ExcelWriter('diccionario_datos.xlsx')
dict_df.to_excel(dicc_datos, sheet_name='Datos', header=True, index=False)
pd.DataFrame(mun_dict.items(), columns=['cve_municipio', 'Municipio'])\
    .to_excel(dicc_datos, sheet_name='Municipios', header=True, index=False)
dicc_datos.save() 